In [1]:
from __future__ import division
import argparse
import numpy as np
import scipy as sp
from scipy.spatial.distance import cosine
import gzip

# Embedding for Medical Concept

Results from paper
<b> Y. Choi, Y. Chiu, D. Sontag. <a href="http://cs.nyu.edu/~dsontag/papers/ChoiChiuSontag_AMIA_CRI16.pdf">Learning Low-Dimensional Representations of Medical Concepts</a>. Published in Proceedings of the AMIA Summit on Clinical Research Informatics (CRI), 2016.</b>

 `claims_codes_hs_300.txt.gz`: Embeddings of ICD-9 diagnosis and procedure codes, NDC medication codes, and LOINC laboratory codes, derived from a large claims dataset from 2005 to 2013 for roughly 4 million people.



### Load the CUI, ICD9, NDC, CPT, LOINIC mapping 

In [2]:
#read in the cui embedding 
cuifile = 'claims_cuis_hs_300.txt.gz'
cui_embeddings = {}
with gzip.open(cuifile, 'r') as infile:
    data = infile.readlines()
    for row in data:
        eles = row.decode('utf8').strip().split(' ')
        name = eles[0]
        embedding = ' '.join(eles[1:])
        cui_embeddings[name] = embedding

In [633]:

ie = 'claims_codes_hs_300.txt.gz'

ndc_to_embeddings = {}
loinic_to_embedding = {}
cpt_to_embedding = {}
icd9_to_embedding = {}

with gzip.open(ie, 'r') as infile:
    data = infile.readlines()
    for row in data:
        eles = row.decode('utf8').strip().split(' ')
        name = eles[0]
        embedding = ' '.join(eles[1:])
        if name[0] == 'N':
            ndc_to_embeddings[name[2:]] = embedding
        if name[0] == 'L':
            loinic_to_embedding[name[2:]] = embedding
        elif name[0] == 'C':
            cpt_to_embedding[name[2:]] = embedding           
        if name[0] == 'I':
            icd9_to_embedding[name[4:]] = embedding


In [634]:
print ('lenght of LOINIC dictionary: ', len(loinic_to_embedding))
print ('lenght of ICD9 dictionary: ' ,len(icd9_to_embedding))
print ('lenght of NDC dictionary: ' ,len(ndc_to_embeddings))
print ('lenght of CPT dictionary: ' ,len(cpt_to_embedding))

lenght of LOINIC dictionary:  3093
lenght of ICD9 dictionary:  14922
lenght of NDC dictionary:  21565
lenght of CPT dictionary:  11746


In [641]:
n_dim = 300

#write the Lab embeddings  
outfilename = 'loinic_to_embedding.txt'
with open(outfilename, 'w') as outfile:
    outfile.write('%s %s\n' %(len(loinic_to_embedding), n_dim))
    for lab in loinic_to_embedding:
        outfile.write('%s %s\n' %(lab, loinic_to_embedding[lab]))
        
#write the ICD9 embeddings  
outfilename = 'icd9_embedding.txt'
with open(outfilename, 'w') as outfile:
    outfile.write('%s %s\n' %(len(icd9_to_embedding), n_dim))
    for icd in icd9_to_embedding:
        outfile.write('%s %s\n' %(icd, icd9_to_embedding[icd]))
        
#write the NDC embeddings  
outfilename = 'ndc_embeddings.txt'
with open(outfilename, 'w') as outfile:
    outfile.write('%s %s\n' %(len(ndc_to_embeddings), n_dim))
    for ndc in ndc_to_embeddings:
        outfile.write('%s %s\n' %(ndc, ndc_to_embeddings[ndc]))
        
#write the ICD9 embeddings  
outfilename = 'cpt_embedding.txt'
with open(outfilename, 'w') as outfile:
    outfile.write('%s %s\n' %(len(cpt_to_embedding), n_dim))
    for cpt in cpt_to_embedding:
        outfile.write('%s %s\n' %(cpt, cpt_to_embedding[cpt]))        

### convert of the ndc and icd to cui

In [53]:
#load the mapping files 
# using parents with smallest number of child
ingredient_to_ndcs = {}
with open('doc/ingredient_ndcs.txt', 'r') as infile:
    data = infile.readlines()
    for row in data:
        eles = row.strip().split(' ')
        ingredient_to_ndcs[eles[0]] = eles[1:]

# using parents with smallest number of child
ingredient_to_ndcs = {}
with open('doc/ingredient_ndcs.txt', 'r') as infile:
    data = infile.readlines()
    for row in data:
        eles = row.strip().split(' ')
        ingredient_to_ndcs[eles[0]] = eles[1:]

# using all parents --- this is the method of converting NDCs to CUIs
# that we are using for this paper. The below are used in the paper.
ingredient_to_all_ndcs = {}
with open('doc/ingredient_all_ndcs.txt', 'r') as infile:
    data = infile.readlines()
    for row in data:
        eles = row.strip().split(' ')
        ingredient_to_all_ndcs[eles[0]] = eles[1:]
        
        
def get_icd9_cui_mappings():
    cui_to_icd9 = {}
    icd9_to_cui = {}
    with open('doc/cui_icd9.txt', 'r') as infile:
        data = infile.readlines()
        for row in data:
            ele = row.strip().split('|')
            if ele[11] == 'ICD9CM':
                cui = ele[0]
                icd9 = ele[10]
                if cui not in cui_to_icd9 and icd9 != '' and '-' not in icd9:
                    cui_to_icd9[cui] = icd9
                    icd9_to_cui[icd9] = cui
    return cui_to_icd9, icd9_to_cui


In [66]:
cui_to_icd9, icd9_to_cui   =  get_icd9_cui_mappings()
print ('size of cui to icd9 mapping: ', len(cui_to_icd9))
print ('size of icd9 to cui mapping: ', len(icd9_to_cui))

size of cui to icd9 mapping:  20839
size of icd9 to cui mapping:  20839


In [89]:
ie = 'claims_codes_hs_300.txt'

ndc_to_embeddings = {}
icd9_cui_to_embeddings = {}
loinic_to_embedding = {}
cpt_to_embedding = {}
icd9_to_embedding = {}
with open(ie, 'r') as infile:
    data = infile.readlines()
    for row in data:
        eles = row.strip().split(' ')
        name = eles[0]
        embedding = ' '.join(eles[1:])
        if name[0] == 'N':
            ndc_to_embeddings[name[2:]] = embedding
        if name[0] == 'L':
            loinic_to_embedding[name[2:]] = embedding
        elif name[0] == 'C':
            cpt_to_embedding[name[2:]] = embedding           
        if name[0] == 'I':
            icd9_to_embedding[name[4:]] = embedding
            if name[4:] in icd9_to_cui:
                cui = icd9_to_cui[name[4:]]
                icd9_cui_to_embeddings[cui] = embedding


In [92]:
#map of the ndc to cui 
ndc_embeddings = []
for ingredient in ingredient_to_all_ndcs.keys():
    ndcs = ingredient_to_all_ndcs[ingredient]
    embeddings = []
    for ndc in ndcs:
        if ndc in ndc_to_embeddings:
            embedding = np.array(ndc_to_embeddings[ndc].split(' '), dtype = float)
            embeddings.append(embedding)
    if len(embeddings) > 0:
        embeddings = np.array(embeddings)
        embedding = np.mean(embeddings, axis=0)
        ndc_embeddings.append((ingredient, embedding))
    else:
        print ('not found')

not found
not found
not found
not found
not found
not found
not found


### Map of the Concept ID (CUI) to ICD 
Similar mapping for the types of ['MSH', 'MTH', 'MEDLINEPLUS', 'NCI_FDA', 'NCI_NICHD', 'NCI', 'CST', 'MDR',
 'ICPC2P', 'ICPC2ICD10ENG', 'CCPSS', 'COSTAR', 'DXP', 'WHO', 'LCH', 'RCD', 'RCDSY',
'SNM', 'OMIM', 'LNC', 'SNOMEDCT_US', 'LCH_NW', 'HPO', 'NANDA-I','NOC', 'ICD9CM', 'BI',
 'CHV', 'MEDCIN', 'SNMI', 'ICPC2EENG', 'ICF-CY', 'ICF', 'ICD10AM', 'ICD10CM', 'ICD10',
 'SNOMEDCT_VET','CCS','CCS_10', 'NCI_NCI-GLOSS']

In [16]:
#read in the cui embedding 
cuifile = 'claims_cuis_hs_300.txt.gz'
cui_embeddings = {}
with gzip.open(cuifile, 'r') as infile:
    data = infile.readlines()
    for row in data:
        eles = row.decode('utf8').strip().split(' ')
        name = eles[0]
        embedding = ' '.join(eles[1:])
        cui_embeddings[name] = embedding

#### ICD9 

In [522]:
cui_to_icd9cm = {}
icd9cm_to_cui = {}
with open('MRCONSO.RRF', 'r') as infile:
    lines = infile.readlines()
    for row in lines:
        ele = row.strip().split('|')
        if (ele[1] == 'ENG') & (ele[11] in ['ICD9CM']) & (ele[0] not in cui_to_icd9cm) & (ele[10]!= '') & ('-' not in ele[10]): 
            icd = ele[10] 
            cui_to_icd9cm[ele[0]] = icd
            if (icd not in icd9cm_to_cui):
                icd9cm_to_cui[icd] = [ele[0]]
            else: 
                if ele[0] not in icd9cm_to_cui[icd]:
                    icd9cm_to_cui[icd].append(ele[0])

In [523]:
print ('size of cui to icd9cm mapping: ', len(cui_to_icd9cm))
print ('size of icd9cm to cui mapping: ', len(icd9cm_to_cui))

size of cui to icd9cm mapping:  20849
size of icd9cm to cui mapping:  20849


In [524]:
icd9cm_to_embedding={}
for icd in icd9cm_to_cui.keys():
    cuis = icd9cm_to_cui[icd]
    embeddings = []
    for cui in cuis:
        if cui in cui_embeddings.keys():
            embedding = np.array(cui_embeddings[cui].split(' '), dtype = float)
            embeddings.append(embedding)
    if len(embeddings) > 0 :
        embeddings = np.array(embeddings)
        embedding = np.mean(embeddings, axis=0)
        icd9cm_to_embedding[icd] = embedding

In [525]:
print( 'size of CUI embedding: ', len(cui_embeddings))
print( 'size of embedding ICD9 (from the paper): ', len(icd9_to_embedding))
print( 'size of embedding ICD9CM: ', len(icd9cm_to_embedding))

size of CUI embedding:  14853
size of embedding ICD9 (from the paper):  14922
size of embedding ICD9CM:  13867


In [630]:
#write the ICD9 embeddings  
outfilename = 'icd9cm_embedding.txt'
with open(outfilename, 'w') as outfile:
    outfile.write('%s %s\n' %(len(icd9cm_to_embedding), embedding.shape[0]))
    for icd in icd9cm_to_embedding:
        outfile.write('%s %s\n' %(icd, icd9cm_to_embedding[icd]))

#### ICD10

In [106]:
#genereate the mapping of the CUI and ICD10CM 
cui_to_icd10cm = {}
icd10cm_to_cui = {}
with open('MRCONSO.RRF', 'r') as infile:
    lines = infile.readlines()
    for row in lines:
        ele = row.strip().split('|')
        if (ele[1] == 'ENG') & (ele[11] in ['ICD10CM']) & (ele[0] not in cui_to_icd10cm) & (ele[13]!= '') & ('-' not in ele[13]):
            cui_to_icd10cm[ele[0]] = ele[13] 
            if (ele[13] not in icd10cm_to_cui):
                icd10cm_to_cui[ele[13]] = [ele[0]]
            else: 
                if ele[0] not in icd10cm_to_cui[ele[13]]:
                    icd10cm_to_cui[ele[13]].append(ele[0] )

In [74]:
print('size of icd10 to cui mapping: ', len(icd10cm_to_cui))
print('size of cui to icd10 mapping: ', len(cui_to_icd10cm))

size of icd10 to cui mapping:  92497
size of cui to icd10 mapping:  102355


In [50]:
icd10cm_to_embedding={}

for icd in icd10cm_to_cui.keys():
    cuis = icd10cm_to_cui[icd]
    embeddings = []
    for cui in cuis:
        if cui in cui_embeddings.keys():
            embedding = np.array(cui_embeddings[cui].split(' '), dtype = float)
            embeddings.append(embedding)
    if len(embeddings) > 0 :
        embeddings = np.array(embeddings)
        embedding = np.mean(embeddings, axis=0)
        icd10cm_to_embedding[icd] = embedding

In [97]:
print( 'size of CUI embedding: ', len(cui_embeddings))
print( 'size of embedding ICD10CM: ', len(icd10cm_to_embedding))

size of CUI embedding:  14853
size of embedding ICD10CM:  4394


In [643]:
ndim = 300
#write the ICD10 embeddings  
outfilename = 'icd10_embedding.txt'
with open(outfilename, 'w') as outfile:
    outfile.write('%s %s\n' %(len(icd10cm_to_embedding), ndim))
    for icd in icd10cm_to_embedding:
        outfile.write('%s %s\n' %(icd, icd10cm_to_embedding[icd]))

### Map of the Concept ID (CUI) to CCS  (for SUB_SUB_HCUP code) 

#### directly from the CUI to CCS 

In [170]:
cui_to_ccs = {}
ccs_to_cui = {}
ccs_dict = {}
with open('MRCONSO.RRF', 'r') as infile:
    lines = infile.readlines()
    for row in lines:
        ele = row.strip().split('|')
        if (ele[1] == 'ENG') & (ele[11] in ['CCS']) & (ele[0] not in cui_to_ccs) & (ele[12]== 'SD') & ('.' not in ele[13]):
            cui_to_ccs[ele[0]] = ele[13] 
            ccs_dict[ele[13]] = ele[14]
            if (ele[13] not in ccs_to_cui):
                ccs_to_cui[ele[13]] = [ele[0]]
            else: 
                if ele[0] not in ccs_to_cui[ele[13]]:
                    ccs_to_cui[ele[13]].append(ele[0] )

In [179]:
print('size of ccs to cui mapping: ', len(ccs_to_cui))
print('size of cui to ccs mapping: ', len(cui_to_ccs))
print('size of ccs mapping: ', len(ccs_dict))

size of ccs to cui mapping:  280
size of cui to ccs mapping:  280
size of ccs mapping:  280


In [173]:
ccs_to_embedding={}
for ccs in ccs_to_cui.keys():
    cuis = ccs_to_cui[ccs]
    embeddings = []
    for cui in cuis:
        if cui in cui_embeddings.keys():
            embedding = np.array(cui_embeddings[cui].split(' '), dtype = float)
            embeddings.append(embedding)
    if len(embeddings) > 0 :
        embeddings = np.array(embeddings)
        embedding = np.mean(embeddings, axis=0)
        ccs_to_embedding[ccs] = embedding

In [267]:
print('size of ccs embedding: ', len(ccs_to_embedding))

size of ccs embedding:  75


#### map from icd9 to ccs 

In [593]:
icd9_hcup_map = pd.read_csv('doc/ICD9_HCUP_MAP.csv')
icd9_hcup_map['diag_part2'] = icd9_hcup_map[['diag_cd', 'diag_firstpart']].apply(lambda x: x.diag_cd[len(x.diag_firstpart):], axis = 1)
icd9_hcup_map['diag_firstpart'] = icd9_hcup_map['diag_firstpart'].apply(lambda x: x.zfill(3))
icd9_hcup_map['diag_code'] = icd9_hcup_map[['diag_firstpart', 'diag_part2']].apply(lambda x: x.diag_firstpart  if not x.diag_part2  else 
                                                                                   x.diag_firstpart + '.' + x.diag_part2  , axis = 1)

In [609]:
icd9_hcup_map['sub_sub_hcup_cd'] = icd9_hcup_map['sub_sub_hcup_cd'].astype(str)
icd9_to_hcup = icd9_hcup_map[['diag_code','sub_sub_hcup_cd']].set_index('diag_code').T.to_dict('list')
hcup_to_icd9 = icd9_hcup_map[['diag_code','sub_sub_hcup_cd']].groupby('sub_sub_hcup_cd')['diag_code'].apply(list).to_frame()
hcup_to_icd9 = hcup_to_icd9.T.to_dict('list')

/Users/yandanyang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  


In [610]:
print('icd9 to ccs size:', len(icd9_to_hcup))
print('ccs to icd9 size:', len(hcup_to_icd9))

icd9 to ccs size: 14423
ccs to icd9 size: 282


In [617]:
ccs_embedding={}
for ccs in hcup_to_icd9.keys():
    icd9 = hcup_to_icd9[ccs][0]
    embeddings = []
    for icd in icd9:
        if icd in icd9cm_to_embedding.keys():
            embedding = icd9cm_to_embedding[icd]
            embeddings.append(embedding)
    if len(embeddings) > 0 :
        embeddings = embeddings
        embedding = np.mean(embeddings, axis=0)
        ccs_embedding[ccs] = embedding

In [618]:
print('size of ccs embedding: ', len(ccs_embedding))

size of ccs embedding:  279


In [626]:
#write the CUI embeddings  
outfilename = 'ccs_embedding.txt'
with open(outfilename, 'w') as outfile:
    outfile.write('%s %s\n' %(len(ccs_embedding), embedding.shape[0]))
    for ccs in ccs_embedding:
        outfile.write('%s %s\n' %(ccs, ccs_embedding[ccs]))

### ICD9-CUI-CCS 

In [479]:
# generate ccs to icd9 mapping 
ccs_to_desc = {}
ccs_to_icd9 = {}
with open('doc/ccs_icd9.txt', 'r') as infile:
    data = infile.readlines()
    currect_ccs = ''
    for row in data:
        if row[:5].strip() != '':
            current_ccs = row[:5].strip()
            ccs_to_desc[current_ccs] = row[5:].strip()
            ccs_to_icd9[current_ccs] = set([])
        elif row.strip() != '':
            ccs_to_icd9[current_ccs].update(set(row.strip().split(' ')))                    

## Concept Mapping 

### an example, map by the names manually 

In [286]:
cui_list = cui_embeddings.keys()
MRC_names = {}
with open('MRCONSO.RRF', 'r') as infile:
    lines = infile.readlines()
    for row in lines:       
        ele = row.strip().split('|')
        if (ele[1] == 'ENG') & (ele[0] not in MRC_names) :
            MRC_names[ele[0]] = ele[14]  
            
cui_names = {}
for x in cui_list:
    if x in MRC_names.keys():
        cui_names[x] = MRC_names[x]            

In [301]:
cui_names = {}
for x in cui_list:
    if x in MRC_names.keys():
        cui_names[x] = MRC_names[x].lower()          

In [310]:
ls = []
for x in cui_names.keys():
    if ('breast' in cui_names[x]) & ('neoplasm' in cui_names[x]):
        ls.append(x)

In [314]:
for i in ls:
    if i in cui_names.keys():
        print(i, cui_names[i])

C1455990 encounter due to genetic susceptibility to malignant neoplasm of breast
C0346156 benign neoplasm of breast
C1261325 family history of malignant neoplasm of breast
C0153555 malignant neoplasm of other specified sites of female breast
C0153558 malignant neoplasm of nipple and areola of male breast
C0153559 malignant neoplasm of other and unspecified sites of male breast
C0153554 malignant neoplasm of axillary tail of female breast
C0153550 malignant neoplasm of upper-inner quadrant of female breast
C0153551 malignant neoplasm of lower-inner quadrant of female breast
C0153552 malignant neoplasm of upper-outer quadrant of female breast
C0153553 malignant neoplasm of lower-outer quadrant of female breast
C0260421 personal history of malignant neoplasm of breast
C0235653 malignant neoplasm of female breast
C0024621 malignant neoplasm of nipple and areola of female breast
C0027641 neoplasm of unspecified nature of breast (disorder)
C2921311 screening examination for malignant neoplas

In [364]:
cui_icd_ccs = {}
cui_sub = {}
for i in cui_names:
    if i in cui_to_icd9:
        if cui_to_icd9[i] in icd9_to_hcup:
            cui_sub[i] = cui_names[i]
            cui_icd_ccs[i] = icd9_to_hcup[cui_to_icd9[i]]            

# Embedding from UMLS 

`stanford_cuis_svd_300.txt.gz`: Embeddings of [UMLS](https://www.nlm.nih.gov/research/umls/) concept unique identifiers (CUIs), derived from 20 million clinical notes spanning 19 years of data from Stanford Hospital and Clinics, using a  [data set](http://datadryad.org/resource/doi:10.5061/dryad.jp917) released in a [paper](http://www.nature.com/articles/sdata201432) by Finlayson, LePendu & Shah.



In [196]:
#read in the cui embedding 
svdfile = 'stanford_cuis_svd_300.txt.gz'
svd_embeddings = {}
with gzip.open(svdfile, 'r') as infile:
    data = infile.readlines()
    for row in data:
        eles = row.decode('utf8').strip().split(' ')
        name = eles[0]
        embedding = ' '.join(eles[1:])
        svd_embeddings[name] = embedding
        
#Read in the concept_to_CUI map
concept_to_CUI_hdr = 'doc/2b_concept_ID_to_CUI.txt'
concept_to_CUI_map = {}
f = open(concept_to_CUI_hdr, 'r')
lines = f.readlines()
for line in lines:
    concept = line.split('\t')[0]
    CUI = line.split('\t')[1].split('\r')[0].splitlines()
    concept_to_CUI_map[concept] = CUI

cui_svd_embeddings = {}
for  line in svd_embeddings.keys():
    if line in concept_to_CUI_map.keys():
        cui = concept_to_CUI_map[line][0]
        cui_svd_embeddings[cui] = svd_embeddings[line]    

In [198]:
icd10cm_svd_embedding={}

for icd in icd10cm_to_cui.keys():
    cuis = icd10cm_to_cui[icd]
    embeddings = []
    for cui in cuis:
        if cui in cui_svd_embeddings.keys():
            embedding = np.array(cui_svd_embeddings[cui].split(' '), dtype = float)
            embeddings.append(embedding)
    if len(embeddings) > 0 :
        embeddings = np.array(embeddings)
        embedding = np.mean(embeddings, axis=0)
        icd10cm_svd_embedding[icd] = embedding

In [199]:
print( 'size of CUI embedding: ', len(svd_embeddings))
print( 'size of embedding ICD10CM: ', len(icd10cm_svd_embedding))

size of CUI embedding:  22706
size of embedding ICD10CM:  3040


In [201]:
ccs_to_svd_embedding={}
for ccs in ccs_to_cui.keys():
    cuis = ccs_to_cui[ccs]
    embeddings = []
    for cui in cuis:
        if cui in cui_svd_embeddings.keys():
            embedding = np.array(cui_svd_embeddings[cui].split(' '), dtype = float)
            embeddings.append(embedding)
    if len(embeddings) > 0 :
        embeddings = np.array(embeddings)
        embedding = np.mean(embeddings, axis=0)
        ccs_to_svd_embedding[ccs] = embedding

In [202]:
len(ccs_to_svd_embedding.keys())

103

# Embedding  from Semantic Similarity

Results generated from paper: 

 `DeVine_etal_200.txt.gz`: Embeddings of UMLS CUIs learned by [De Vine et al. CIKM '14](http://dl.acm.org/citation.cfm?id=2661974), derived from 348,566 medical journal abstracts (courtesy of the authors).


In [147]:
#read in the cui embedding 
file = 'DeVine_etal_200.txt.gz'
cui200_embeddings = {}
with gzip.open(file, 'r') as infile:
    data = infile.readlines()
    for row in data:
        eles = row.decode('utf8').strip().split(' ')
        name = eles[0]
        embedding = ' '.join(eles[1:])
        cui200_embeddings[name] = embedding

In [193]:
icd10cm_DeVine_embedding={}

for icd in icd10cm_to_cui.keys():
    cuis = icd10cm_to_cui[icd]
    embeddings = []
    for cui in cuis:
        if cui in cui200_embeddings.keys():
            embedding = np.array(cui200_embeddings[cui].split(' '), dtype = float)
            embeddings.append(embedding)
    if len(embeddings) > 0 :
        embeddings = np.array(embeddings)
        embedding = np.mean(embeddings, axis=0)
        icd10cm_DeVine_embedding[icd] = embedding

In [196]:
print( 'size of CUI embedding: ', len(cui200_embeddings))
print( 'size of embedding ICD10CM: ', len(icd10cm_DeVine_embedding))

size of CUI embedding:  52103
size of embedding ICD10CM:  3995


# Check the similarity 

In [199]:
# find the common icd10 
icd_ls = []
for x in icd10cm_to_embedding.keys():
    if x in icd10cm_svd_embedding.keys():
        icd_ls.append(x)

In [203]:
for x in icd_ls[:20]:
    print(cosine(icd10cm_to_embedding[x],icd10cm_svd_embedding[x]))

1.0281133579317911
0.9653097554607961
0.8929884075824519
1.0250823382441114
1.0351952733075056
1.0134070325694438
0.9385475067068678
0.9235336549321366
0.8525211368621229
1.0208316444842074
0.9565962753014222
1.0471509774815773
1.0578278027958175
0.9626119194801128
1.0699642739287951
1.058134600588414
0.972127745445609
0.9520061647935075
1.0166671695986844
0.9633936535634939
